# Oracle SQL*Plus in IPython/Jupyter notebooks
- A simple example of how to query Oracle from a Jupyter notebook
- It runs sqlplus (oracle command line) using %%bash magic and [here documents]([https://en.wikipedia.org/wiki/Here_document)
- Dependencies: needs an Oracle client installation
- Pros: easy to implement
- Cons: does not provide a true integration with the Python environemnt, see the examples with cx_Oracle to see how that can be done.

## A basic example to get started

In [1]:
%%bash
sqlplus -s test/test@localhost:1521/orcl.cern.ch <<EOF

set serveroutput on
begin
  dbms_output.put_line('Hello world!');
end;
/

select sysdate from dual;

EOF

Hello world!

PL/SQL procedure successfully completed.


SYSDATE
---------
02-JUN-16



## Query from a test table

In [2]:
%%bash
sqlplus -s scott/tiger@localhost:1521/orcl.cern.ch <<EOF

set linesize 100
set pagesize 100
col ename for a15
col job for a15

SELECT * from EMP;

EOF


     EMPNO ENAME	   JOB			  MGR HIREDATE	       SAL	 COMM	  DEPTNO
---------- --------------- --------------- ---------- --------- ---------- ---------- ----------
      7902 FORD 	   ANALYST		 7566 03-DEC-81       3000		      20
      7844 TURNER	   SALESMAN		 7698 08-SEP-81       1500	    0	      30
      7698 BLAKE	   MANAGER		 7839 01-MAY-81       2850		      30
      7654 MARTIN	   SALESMAN		 7698 28-SEP-81       1250	 1400	      30
      7788 SCOTT	   ANALYST		 7566 19-APR-87       3000		      20
      7566 JONES	   MANAGER		 7839 02-APR-81       2975		      20
      7369 SMITH	   CLERK		 7902 17-DEC-80        800		      20
      7839 KING 	   PRESIDENT		      17-NOV-81       5000		      10
      7521 WARD 	   SALESMAN		 7698 22-FEB-81       1250	  500	      30
      7934 MILLER	   CLERK		 7782 23-JAN-82       1300		      10
      7499 ALLEN	   SALESMAN		 7698 20-FEB-81       1600	  300	      30
      7900 JAMES	   CLERK		 7698 03-DEC-81        950		      30
      7782 CLARK	   

## An example with some funky SQL

In [3]:
%%bash
sqlplus -s test/test@localhost:1521/orcl.cern.ch <<EOF

set verify off
set lines 4000
set pages 999
set heading off

-- From https://github.com/LucaCanali/Miscellaneous/tree/master/SQL_color_Mandelbrot
-- Configuration parameters for the Mandelbrot set calculation
-- Edit to change the region displayed and/or resolution by changing the definitions here below
-- Edit your terminal screen resolution and/or modify XPOINTS and YPOINTS so that the image fits the screen
define XMIN=-2.0
define YMIN=-1.4
define XMAX=0.5
define YMAX=1.4
define XPOINTS=120
define YPOINTS=60
define XSTEP="(&XMAX - &XMIN)/(&XPOINTS - 1)"
define YSTEP="(&YMAX - &YMIN)/(&YPOINTS - 1)"

-- Visualization parameters 
define COLORMAP="012223333344445555666677770"
define MAXITER="LENGTH('&COLORMAP')"
define BLUE_PALETTE="0,0,1,15,2,51,3,45,4,39,5,33,6,27,7,21"
define PALETTE_NUMCOLS=8
define ESCAPE_VAL=4
define ANSICODE_PREFIX="chr(27)||'[48;5;'"
define ANSICODE_BACKTONORMAL="chr(27)||'[0m'"

WITH
   XGEN AS (                            -- X dimension values generator
        SELECT CAST(&XMIN + &XSTEP * (rownum-1) AS binary_double) AS X, rownum AS IX FROM DUAL CONNECT BY LEVEL <= &XPOINTS),
   YGEN AS (                            -- Y dimension values generator
        SELECT CAST(&YMIN + &YSTEP * (rownum-1) AS binary_double) AS Y, rownum AS IY FROM DUAL CONNECT BY LEVEL <= &YPOINTS),
   Z(IX, IY, CX, CY, X, Y, I) AS (     -- Z point iterator. Makes use of recursive common table expression 
        SELECT IX, IY, X, Y, X, Y, 0 FROM XGEN, YGEN
        UNION ALL
        SELECT IX, IY, CX, CY, X*X - Y*Y + CX, 2*X*Y + CY, I+1 FROM Z WHERE X*X + Y*Y < &ESCAPE_VAL AND I < &MAXITER),
   MANDELBROT_MAP AS (                       -- Computes an approximated map of the Mandelbrot set
        SELECT IX, IY, MAX(I) AS VAL FROM Z  -- VAL=MAX(I) represents how quickly the values reached the escape point
        GROUP BY IY, IX),
   PALETTE AS (                              -- Color palette generator using ANSI escape codes
        SELECT rownum-1 ID, &ANSICODE_PREFIX|| DECODE(rownum-1, &BLUE_PALETTE) || 'm ' || &ANSICODE_BACKTONORMAL COLOR 
        FROM DUAL CONNECT BY LEVEL <= &PALETTE_NUMCOLS)
SELECT LISTAGG(PALETTE.COLOR) WITHIN GROUP (ORDER BY IX) GRAPH        -- The function LISTAGG concatenates values into rows
FROM MANDELBROT_MAP, PALETTE
WHERE TO_NUMBER(SUBSTR('&COLORMAP',MANDELBROT_MAP.VAL,1))=PALETTE.ID  -- Map visualization using PALETTE and COLORMAP
GROUP BY IY
ORDER BY IY DESC;

EOF
